# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [3]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [4]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [5]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [6]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [7]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [8]:
create_keyspace = "CREATE KEYSPACE IF NOT EXISTS music_library WITH replication = {'class': 'SimpleStrategy', 'replication_factor' : '3'}"
session.execute(create_keyspace)


#### Set Keyspace

In [9]:
# TO-DO: Set KEYSPACE to the keyspace specified above
session.set_keyspace("music_library")


### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [10]:
create_table1 = "CREATE TABLE IF NOT EXISTS played_songs_by_session (session_id int, item_in_session int, artist_name text, song_title text, song_length float, PRIMARY KEY(session_id, item_in_session))"
create_table2 = "CREATE TABLE IF NOT EXISTS played_songs_by_user (user_id int, session_id int, item_in_session int, artist_name text, song_title text, user_fname text, user_lname text, PRIMARY KEY((user_id, session_id), item_in_session))"
create_table3 = "CREATE TABLE IF NOT EXISTS user_detail_by_song (song_title text, artist_name text, user_id int, user_fname text, user_lname text, PRIMARY KEY(song_title, artist_name, user_id))"
session.execute(create_table1)
session.execute(create_table2)
session.execute(create_table3)




                    

In [11]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#

insert_songs_by_session = "INSERT INTO played_songs_by_session (session_id, item_in_session, artist_name, song_title, song_length) VALUES (%s, %s, %s, %s, %s)"
insert_songs_by_user = "INSERT INTO played_songs_by_user (user_id, session_id, item_in_session, artist_name, song_title, user_fname, user_lname) VALUES (%s, %s, %s, %s, %s, %s, %s)"
insert_users_by_song = "INSERT INTO user_detail_by_song (song_title, artist_name, user_id, user_fname, user_lname) VALUES (%s, %s, %s, %s, %s)"

file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        session.execute(insert_songs_by_session, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))
        session.execute(insert_songs_by_user, (int(line[10]), int(line[8]),int(line[3]), line[0], line[9], line[1], line[4]))
        session.execute(insert_users_by_song, (line[9], line[0], int(line[10]), line[1], line[4]))


In [ ]:
### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [13]:
# Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4
# Partition key: session_id  - To  data gets distributed evenly for this query 
# Clustering key: item_in_session 
select_query1 = "SELECT artist_name, song_title, song_length from played_songs_by_session WHERE session_id = {} AND item_in_session = {}".format(338, 4)
rows = session.execute(select_query1)
print("Result of Query #1")
for row in rows:
    print(row)                    

Result of Query #1
Row(artist_name='Faithless', song_title='Music Matters (Mark Knight Dub)', song_length=495.30731201171875)


In [14]:
# Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
# Partition keys: user_id & session_id - To data gets distributed evenly for this query
# Clustering key: session_id 
select_query2 = "SELECT artist_name, song_title, user_fname, user_lname from played_songs_by_user WHERE user_id = {} AND session_id = {}".format(10, 182)
rows = session.execute(select_query2)
for row in rows:
    print(row)
                    

Row(artist_name='Down To The Bone', song_title="Keep On Keepin' On", user_fname='Sylvie', user_lname='Cruz')
Row(artist_name='Three Drives', song_title='Greece 2000', user_fname='Sylvie', user_lname='Cruz')
Row(artist_name='Sebastien Tellier', song_title='Kilometer', user_fname='Sylvie', user_lname='Cruz')
Row(artist_name='Lonnie Gordon', song_title='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', user_fname='Sylvie', user_lname='Cruz')


In [15]:
# Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
# Partition key: song_title   - To data gets distributed evenly  
# Clustering keys: artist_name, user_id
select_query3 = "SELECT user_fname, user_lname from user_detail_by_song WHERE song_title = 'All Hands Against His Own'"
rows = session.execute(select_query2)
for row in rows:
    print(row)

Row(artist_name='Down To The Bone', song_title="Keep On Keepin' On", user_fname='Sylvie', user_lname='Cruz')
Row(artist_name='Three Drives', song_title='Greece 2000', user_fname='Sylvie', user_lname='Cruz')
Row(artist_name='Sebastien Tellier', song_title='Kilometer', user_fname='Sylvie', user_lname='Cruz')
Row(artist_name='Lonnie Gordon', song_title='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', user_fname='Sylvie', user_lname='Cruz')


### Drop the tables before closing out the sessions

In [16]:
drop_table1 =  "DROP TABLE IF EXISTS played_songs_by_session"
drop_table2 =  "DROP TABLE IF EXISTS played_songs_by_user"
drop_table3 =  "DROP TABLE IF EXISTS user_detail_by_song"
session.execute(drop_table1)
session.execute(drop_table2)
session.execute(drop_table3)

### Close the session and cluster connection¶

In [17]:
session.shutdown()
cluster.shutdown()